In [ ]:
import pandas as pd
import pickle 
import numpy as np
import os
import sys
sys.path.append('/home/stankeaa/wikiwho_inheritance/')
from sequencers.vectorizers import bert_lr, bert_lrg, distilbert_lr, distilbert_lrg, word_embed 
import ipywidgets as widgets
from IPython.display import display, clear_output
import re
from os.path import join

In [ ]:
dirs = []
for file in os.listdir( sys.path[-1] ):
    if re.match("^[0-9_-]*$", file):
        dirs.append(file)
vecs = ['bert_lr.Bert_LR', 'bert_lrg.Bert_LRG', 'distilbert_lr.Distilbert_LR', 'distilbert_lrg.Distilbert_LRG', 
        'word_embed.Word_Embed']

## Vectorizers
This notebook adds vector features to wikiwho chobs. It requires an **existing pickle** with processed chobs dataframe. <br>
There are different vectorizers possible: <br>
Bert (either only left and right context - Bert_LR - or left and right context with gap - Bert_LRG)<br>
DistilBert (also with only context or with gap as well)<br>
Word Embeddence with following parameters:
1. word embeddence size (length of features vectors)
2. use of gap
3. context size (cannot be larger than in chobs dataframe)

In [ ]:
a = widgets.Dropdown(
    options = dirs,
    value = None,
    description = 'Article id:',
)
pickle = widgets.Dropdown(
    options = [],
    description = 'Chobs pickle:',
)

v = widgets.Dropdown(
    options = vecs,
    description = 'Vectorizer:',
)
word_embed_size = widgets.IntText(
    value=300,
    description='Word embed size:'
)
context = widgets.IntText(
    value=5,
    description='Context:'
)
use_gap = widgets.Checkbox(
    value=True,
    description='Use gap'
)
def a_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        pickle.options = os.listdir(sys.path[-1] + change['new'] + '/chobs/')
        display(pickle, v)

def v_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        global vectorizer
        vectorizer = eval(v.value)
        if change['new'] == 'word_embed.Word_Embed':
            display(word_embed_size, use_gap, context)
        else:
            clear_output()
            display(a, pickle, v)
        print('Data is saved')
            
a.observe(a_on_change)            
v.observe(v_on_change)
display(a)

In [ ]:
vectorizer = eval(v.value)
PICKLE_PATH = sys.path[-1] + a.value + '/chobs/' + pickle.value

if not os.path.exists('../'+a.value +'/vectorizers'):
    os.mkdir('../'+a.value +'/vectorizers')
if vectorizer == word_embed.Word_Embed:
    model = vectorizer(pd.read_pickle(PICKLE_PATH), 
                       {'word_embed_size':word_embed_size.value, 
                              'use_gap':use_gap.value,
                              'context':context.value,
                       '_':pickle.value[:-4]}, 
                       '../'+a.value +'/vectorizers')
else:
    model = vectorizer(pd.read_pickle(PICKLE_PATH), {'_':pickle.value[:-4]}, '../'+a.value +'/vectorizers')
model.get_features()
model.save()